In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Part 1

In [2]:
import os
import pandas as pd

# Define the path to the data directory in your Google Drive
data_dir_path = '/content/drive/MyDrive/AAX/dataset_Seminar5/client_datasets'

# Now, load the data for each client.
client_data = {}
# num_clients = 10 # As specified in the problem description
# Client numbering goes from 1 to 10
client_ids = range(1, 11)


for i in client_ids:
    # Construct the file paths for each client's features and labels
    features_file = os.path.join(data_dir_path, f'client_{i}_features.csv')
    labels_file = os.path.join(data_dir_path, f'client_{i}_labels.csv')

    if os.path.exists(features_file) and os.path.exists(labels_file):
        try:
            # Load the data for the client
            features_df = pd.read_csv(features_file, header=None)
            labels_df = pd.read_csv(labels_file, header=None)

            # Store features and labels for the client
            client_data[f'client_{i}'] = {'features': features_df, 'labels': labels_df}

            print(f"Loaded data for client {i}")
        except Exception as e:
            print(f"Error loading data for client {i}: {e}")
    else:
        print(f"Data files not found for client {i}: {features_file}, {labels_file}")

# Now client_data dictionary contains dataframes for features and labels for each client.
# You can now proceed with preprocessing each client's data as needed for your model.

# Example of how to access a client's data:
# client_1_features_df = client_data['client_1']['features']
# client_1_labels_df = client_data['client_1']['labels']
# display(client_1_features_df.head())
# display(client_1_labels_df.head())

Loaded data for client 1
Loaded data for client 2
Loaded data for client 3
Loaded data for client 4
Loaded data for client 5
Loaded data for client 6
Loaded data for client 7
Loaded data for client 8
Loaded data for client 9
Loaded data for client 10


#Part 2
Implement the Federated Learning setup for the pose estimation problem using the provided dataset. This involves defining the central orchestrator's functionalities: initializing a global model, selecting clients, sending the model to clients for local training, receiving updated models, and aggregating them using the FedAvg algorithm. The dataset is located in the "/content/drive/MyDrive/AAX/dataset_Seminar5/client_datasets/" directory, with separate CSV files for features and labels for each of the 10 clients (client_1_features.csv, client_1_labels.csv, ..., client_10_features.csv, client_10_labels.csv).

## Initialize the global model

### Subtask:
Define and initialize the machine learning model that will be trained collaboratively.


Define a neural network model using PyTorch, which is suitable for the pose estimation task and initialize an instance of the model.



In [3]:
import torch
import torch.nn as nn

class PoseEstimationModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(PoseEstimationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Determine input and output sizes based on the data.
# Assuming features are flattened and labels are one-hot encoded or similar.
# From the data loading step, features_df has shape (64, 270) and labels_df has shape (1, 64).
# The features are likely 270 per sample, and labels are 64 per sample.
# Let's assume input size is 270 and output size is 64.
input_size = 270
num_classes = 64 # Assuming each label column represents a class or a coordinate

global_model = PoseEstimationModel(input_size, num_classes)

print("Global model initialized:")
print(global_model)

Global model initialized:
PoseEstimationModel(
  (fc1): Linear(in_features=270, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
)


## Implement client selection

### Subtask:
Create a mechanism to select a subset of clients for each training round.


Import the `random` module and define the `select_clients` function to randomly select a subset of client IDs.



In [4]:
import random

def select_clients(client_ids, num_selected_clients):
    """
    Selects a random subset of clients for a training round.

    Args:
        client_ids: A list of all available client IDs.
        num_selected_clients: The number of clients to select.

    Returns:
        A list of selected client IDs.
    """
    return random.sample(client_ids, num_selected_clients)

## Implement the fedavg aggregation

### Subtask:
Write code to perform the weighted averaging of client model weights.


Define the `fedavg_aggregate_weights` function to perform weighted averaging of client model weights as per the instructions.



In [5]:
def fedavg_aggregate_weights(client_states):
    """
    Aggregates client model weights using the Federated Averaging (FedAvg) algorithm.

    Args:
        client_states: A list of state dictionaries from client models.

    Returns:
        A state dictionary containing the aggregated weights.
    """
    if not client_states:
        return None

    # Initialize aggregated weights with the first client's weights
    aggregated_weights = {}
    for key in client_states[0]:
        aggregated_weights[key] = client_states[0][key].clone()

    # Sum the weights from the remaining clients
    for i in range(1, len(client_states)):
        for key in client_states[i]:
            aggregated_weights[key] += client_states[i][key]

    # Divide by the total number of clients to get the average
    num_clients = len(client_states)
    for key in aggregated_weights:
        aggregated_weights[key] /= num_clients

    return aggregated_weights

## Simulate the FL training process

### Subtask:
Structure the training loop to simulate the rounds of federated learning, including distributing the global model to selected clients, simulating local training, and performing the FedAvg aggregation.

Implement the federated learning training loop, including client selection, local training simulation, and aggregation using the previously defined functions and loaded data.

In [7]:
import torch
import torch.optim as optim
import random
import torch.nn as nn # Need to re-import nn for the criterion

# Define federated learning parameters
num_rounds = 50  # Number of federated learning rounds
num_clients_per_round = 5 # Number of clients to select per round
local_epochs = 2 # Number of local epochs for each client
local_batch_size = 16 # Batch size for local training
learning_rate = 0.001 # Learning rate for local optimizers

# Get list of all client IDs
all_client_ids = list(range(1, 11))

# Training loop
for round_num in range(num_rounds):
    print(f"--- Federated Learning Round {round_num + 1}/{num_rounds} ---")

    # 3. Select a subset of clients for the current round
    selected_clients_this_round = select_clients(all_client_ids, num_clients_per_round)
    print(f"Selected clients: {selected_clients_this_round}")

    client_states_this_round = []

    # 4. Simulate local training for each selected client
    for client_id in selected_clients_this_round:
        print(f"  Training on client {client_id}")

        # a. Get the current state dictionary from the global_model
        global_model_state = global_model.state_dict()

        # b. Load this state dictionary into a new instance of the PoseEstimationModel (simulating distribution)
        # Re-initialize the model within the loop to simulate distributing a fresh global model
        client_model = PoseEstimationModel(input_size, num_classes)
        client_model.load_state_dict(global_model_state)

        # c. Access the client's data
        client_features_df = client_data[f'client_{client_id}']['features']
        client_labels_df = client_data[f'client_{client_id}']['labels']

        # d. Convert the client's data into PyTorch tensors
        # Ensure labels are in the correct shape (samples, num_classes)
        client_features = torch.tensor(client_features_df.values, dtype=torch.float32)
        client_labels = torch.tensor(client_labels_df.values.T, dtype=torch.float32)


        # Create a TensorDataset and DataLoader for batching
        client_dataset = torch.utils.data.TensorDataset(client_features, client_labels)
        client_dataloader = torch.utils.data.DataLoader(client_dataset, batch_size=local_batch_size, shuffle=True)


        # e. Define a local optimizer and loss function
        client_optimizer = optim.Adam(client_model.parameters(), lr=learning_rate)
        criterion = nn.MSELoss() # Using Mean Squared Error for regression

        # f. Perform local training
        client_model.train() # Set model to training mode
        for epoch in range(local_epochs):
            for batch_features, batch_labels in client_dataloader:
                client_optimizer.zero_grad()
                outputs = client_model(batch_features)
                loss = criterion(outputs, batch_labels)
                loss.backward()
                client_optimizer.step()
            # print(f"    Client {client_id}, Epoch {epoch + 1}/{local_epochs}, Loss: {loss.item():.4f}") # Optional: print local loss


        # g. After local training, get the updated state dictionary
        client_states_this_round.append(client_model.state_dict())
        print(f"  Finished training on client {client_id}")


    # 6. Aggregate the collected client state dictionaries
    print("Aggregating client models...")
    aggregated_global_state = fedavg_aggregate_weights(client_states_this_round)

    # 7. Load the aggregated weights back into the global_model
    global_model.load_state_dict(aggregated_global_state)
    print("Aggregation complete. Global model updated.")

    # 8. (Optional) Print progress or evaluate the global model periodically
    if (round_num + 1) % 10 == 0:
        print(f"--- Global model updated after round {round_num + 1} ---")
        # Add evaluation logic here if needed
        # For example, evaluate on a held-out test set or a validation set
        # This requires defining an evaluation function and having test/validation data.

print("\n--- Federated Learning Training Finished ---")

--- Federated Learning Round 1/50 ---
Selected clients: [3, 4, 9, 1, 5]
  Training on client 3
  Finished training on client 3
  Training on client 4
  Finished training on client 4
  Training on client 9
  Finished training on client 9
  Training on client 1
  Finished training on client 1
  Training on client 5
  Finished training on client 5
Aggregating client models...
Aggregation complete. Global model updated.
--- Federated Learning Round 2/50 ---
Selected clients: [2, 3, 5, 6, 9]
  Training on client 2
  Finished training on client 2
  Training on client 3
  Finished training on client 3
  Training on client 5
  Finished training on client 5
  Training on client 6
  Finished training on client 6
  Training on client 9
  Finished training on client 9
Aggregating client models...
Aggregation complete. Global model updated.
--- Federated Learning Round 3/50 ---
Selected clients: [4, 6, 1, 8, 3]
  Training on client 4
  Finished training on client 4
  Training on client 6
  Finished

## Integrate data loading and preprocessing

### Subtask:
Ensure the previously loaded and preprocessed client data is used within the FL simulation.

Verify and confirm the correct usage of the previously loaded and preprocessed client data within the federated learning simulation loop by examining the relevant parts of the code from the previous turn.

In [8]:
# Verify access to client_data dictionary
print("Verifying access to client_data:")
if 'client_data' in locals() or 'client_data' in globals():
    print("client_data dictionary is accessible.")
    # Confirm presence of data for at least one client
    if 'client_1' in client_data and 'features' in client_data['client_1'] and 'labels' in client_data['client_1']:
        print("Data for 'client_1' found in client_data.")
        print(f"client_1 features type: {type(client_data['client_1']['features'])}")
        print(f"client_1 labels type: {type(client_data['client_1']['labels'])}")
    else:
        print("Data for 'client_1' not found or incomplete in client_data.")
else:
    print("client_data dictionary is not accessible.")

print("\nVerification complete based on code examination.")

Verifying access to client_data:
client_data dictionary is accessible.
Data for 'client_1' found in client_data.
client_1 features type: <class 'pandas.core.frame.DataFrame'>
client_1 labels type: <class 'pandas.core.frame.DataFrame'>

Verification complete based on code examination.
